In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# ============================================================
#  VECTORND (OPERACIONES BÁSICAS)
# ============================================================

class VectorND:
    def __init__(self, coords):
        self.coords = list(coords)

    def copy(self):
        return VectorND(self.coords[:])

    def __len__(self):
        return len(self.coords)

    def __getitem__(self, i):
        return self.coords[i]

    def __repr__(self):
        return f"VectorND({self.coords})"

    def __add__(self, other):
        return VectorND([a + b for a, b in zip(self.coords, other.coords)])

    def __sub__(self, other):
        return VectorND([a - b for a, b in zip(self.coords, other.coords)])

    def __mul__(self, scalar):
        return VectorND([scalar * c for c in self.coords])

    def __rmul__(self, scalar):
        return VectorND([scalar * c for c in self.coords])

In [2]:
# ============================================================
#  RUNGE–KUTTA ORDEN 4
# ============================================================

def rungeKutta(f, y0: VectorND, t0: float, T: float, steps: int, params: dict):
    h = (T - t0) / steps
    t = t0
    y = y0.copy()

    sol = [y.copy()]
    times = [t]

    for _ in range(steps):

        k1 = f(y, t, params)
        k2 = f(y + k1*(h/2), t + h/2, params)
        k3 = f(y + k2*(h/2), t + h/2, params)
        k4 = f(y + k3*h, t + h, params)

        y = y + (k1 + 2*k2 + 2*k3 + k4)*(h/6)
        t += h

        sol.append(y.copy())
        times.append(t)

    return sol, times

In [3]:
# ============================================================
#  POSICIÓN DE LA PARTÍCULA CONDUCIDA (RIGHTMOST)
#  x_drive(t, n_dyn, a) posición inicial en x0 = n_dyn*a.
# ============================================================
def x_drive(t, n_dyn, a=1.0):
    """
    La partícula conductora (driven) empieza en x0 = n_dyn * a
    y se mueve hacia la derecha con un perfil de un polinomio cúbico.
    """
    x0 = n_dyn * a
    return 5 * x0 + (t - (4 * x0)**(1/3))**3 + t

In [4]:
# ============================================================
#  DINÁMICA DE RESORTES ACOPLADOS (RIGHTMOST DRIVEN)
#  y = [x1..x_{n-1}, p1..p_{n-1}]  (left part is dynamic)
# ============================================================
def rhs(y: VectorND, t: float, params: dict) -> VectorND:

    n_total = params["n"]
    m = params["m"]
    k = params["k"]
    a = params["a"]  #Distancia entre partítuclas
    xdrv = params["x_drive"] #Función de la partícula conductora.

    n_dyn = n_total - 1  #Cadena de partículas sin contar la partícula conductora.

    x = y.coords[:n_dyn]   # x1 .. x_{n-1}
    p = y.coords[n_dyn:]   # p1 .. p_{n-1}

    dxdt = [0.0]*n_dyn
    dpdt = [0.0]*n_dyn

    # Velocidades
    for i in range(n_dyn):
        dxdt[i] = p[i] / m

    # Fuerzas
    for i in range(n_dyn):
        force = 0.0

        # spring from left neighbor (if exists)
        if i > 0:
            left = x[i-1]
            force += -k*(x[i] - left - a)

        # spring from right neighbor (or driven rightmost)
        if i < n_dyn - 1:
            right = x[i+1]
            force += k*(right - x[i] - a)
        else:
            # right neighbor is the driven particle (use consistent n_dyn)
            right = xdrv(t, n_dyn, a)
            force += k*(right - x[i] - a)

        dpdt[i] = force

    return VectorND(dxdt + dpdt)

In [13]:
# ============================================================
#  PARÁMETROS E INICIALES
# ============================================================

params = {
    "n": 30,   # total particles including driven rightmost
    "m": 1.0,
    "k": 1.0,
    "a": 5.0,
    "x_drive": x_drive
}

n_dyn = params["n"] - 1

# dynamic particles initially at x = 0, a, 2a, ... (so driven initial is n_dyn*a)
x_init = [params["a"] * i for i in range(n_dyn)]
p_init = [0.0]*n_dyn

y0 = VectorND(x_init + p_init)

In [17]:
# ============================================================
#  INTEGRACIÓN NUMÉRICA
# ============================================================

T = 30
steps = 3000

sol, times = rungeKutta(rhs, y0, 0.0, T, steps, params)


In [18]:
# ============================================================
#  RECONSTRUIR POSICIONES COMPLETAS (dinámicas + conducida)
# ============================================================

full_positions = []
for yvec, t in zip(sol, times):
    x_dyn = yvec.coords[:n_dyn]
    full_positions.append(x_dyn + [params["x_drive"](t, n_dyn, params["a"])])

full_positions = np.array(full_positions)
t2 = np.array(times)
